In [93]:
import pandas as pd
import numpy as np
import copy
import math
from sklearn.preprocessing import LabelEncoder

csv_path = 'Apparel/apparel-trainval.csv'#raw_input("Enter path to input CSV file: ")
dataset = pd.read_csv(csv_path)
# dataset = dataset.truncate(before=0, after=55000)
dataset = dataset.astype('float64')

dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
#We inititalize X and Y as the attributes and label respectively for the decision tree
Attributes = dataset.keys()[1:]
Label = dataset.keys()[0]
print(len(Attributes))
print(Label)

mean = {}
std = {}
for x in Attributes:
    mean[x] = np.mean(dataset[x])
    std[x] = np.std(dataset[x])
    
normalized_dataset = dataset
for x in Attributes:
    normalized_dataset[x] = (normalized_dataset[x] - mean[x]) / std[x]
    
#split data into train data and validation data
splitted = np.split(normalized_dataset, [int(.8 * len(dataset))])
train_data = splitted[0]
validation_data = splitted[1]
    
X = train_data[Attributes].values
# X[X != X] = 0
assert len(X[X != X]) == 0 # No nan should be present
Y = train_data[Label].values

784
label


In [95]:
def one_hot_encode(labels):
#     print labels.shape
    num_labels = len(labels)
    unique = len(np.unique(labels))
    one_hot_encode = np.zeros((num_labels, unique))
    one_hot_encode[np.arange(num_labels), labels] = 1
    return one_hot_encode

In [234]:
class Neural_Network:
    def __init__(self, s1, s2, s3, s4_list, hidden_act, output_act, n):
        # Define Hyperparameters
        self.inputLayerSize = s1
        self.outputLayerSize = s2
        self.hiddenLayerCount = s3
        self.hiddenLayerSizes = s4_list  # s4_list = [hiddenLayer1Size, hiddenLayer2Size, ...]
        self.hiddenActivation = hidden_act  # 'sigmoid', 'relu' or 'tanh'
        self.outputActivation = output_act  # 'sigmoid', 'relu' or 'tanh'
        self.learningRate = n
        
        # Weights (parameters)
        self.W = []
        self.W.append( np.random.rand(self.inputLayerSize, self.hiddenLayerSizes[0]) )
        for i in range(self.hiddenLayerCount-1):
            self.W.append( np.random.rand(self.hiddenLayerSizes[i], self.hiddenLayerSizes[i+1]) )
        self.W.append( np.random.rand(self.hiddenLayerSizes[self.hiddenLayerCount-1], self.outputLayerSize) )
    
        for i in range(len(self.W)):
            self.W[i] = self.W[i] * math.sqrt(2.0/s1)
            
        self.b = [None]*(self.hiddenLayerCount+1)
        for i in range(len(self.b)):
            self.b[i] = np.random.rand()
                
        
    def forward(self, x):
        # Propagate inputs through network
        self.z = []  # z2, z3, ... z(n+1) n=hiddenLayerCount
        self.a = []  # a2, a3, ... a(n+1)
        
        self.z.append( np.dot(x, self.W[0]) )
        
        for i in range(self.hiddenLayerCount):
            cur_a = self.activations[self.hiddenActivation]( self.z[i] )
            self.a.append( cur_a )
            self.z.append( np.dot(cur_a, self.W[i+1]) )
#             print(self.a[i].shape)

#         print self.z[self.hiddenLayerCount].shape
        yHat = self.activations[self.outputActivation]( self.z[self.hiddenLayerCount] )
        
        
#         zz = []
#         aa = []
#         w = self.W[0]+0.00001
#         zz.append( np.dot(x, w) )
#         for i in range(self.hiddenLayerCount):
#             cur_a = self.activations[self.hiddenActivation]( zz[i] )
#             aa.append( cur_a )
#             zz.append( np.dot(cur_a, self.W[i+1]) )
# #             print(self.a[i].shape)
# #             print(self.z[i+1].shape)
#         yHat_test = self.activations[self.outputActivation]( zz[self.hiddenLayerCount] )
#         j = 0.5 * (self.y - yHat_test)**2
#         print self.y.shape, yHat_test.shape, j.shape
        
#         zz = []
#         aa = []
#         w = self.W[0]-0.00001
#         zz.append( np.dot(x, w) )
#         for i in range(self.hiddenLayerCount):
#             cur_a = self.activations[self.hiddenActivation]( zz[i] )
#             aa.append( cur_a )
#             zz.append( np.dot(cur_a, self.W[i+1]) )
# #             print(self.a[i].shape)
# #             print(self.z[i+1].shape)
#         yHat_test = self.activations[self.outputActivation]( zz[self.hiddenLayerCount] )
#         j = j - (  0.5 * (self.y - yHat_test)**2  )
        
#         print 'estimated', (j/0.00002).shape
        
        return yHat
    
    
    def backward2(self, x, y, yHat):
        dJdW = [None] * len(self.W)
#         print len(self.W), len(self.a)
        
        a_idx = len(self.a)-1
        z_idx = len(self.z)-1
#         print(y.shape)
        df_z = self.derivatives['d_'+self.outputActivation]( self.z[z_idx] )
    
        if self.outputLayerSize == 1:
            y = y.reshape(len(y),1)
        
        delta = np.multiply( -(y - yHat), df_z )
        dJdW[ len(dJdW)-1 ] = np.dot( self.a[a_idx].T, delta )
        
        a_idx -= 1
        z_idx -= 1
#         print("-----")
        for i in range(len(dJdW)-2, 0, -1):
            df_z = self.derivatives['d_'+self.hiddenActivation]( self.z[z_idx] )
            print delta.shape, self.W[i+1].shape, df_z.shape
#             print(delta.shape, self.W[i+1].T.shape)
            delta = np.dot(delta, self.W[i+1].T) * df_z
            dJdW[i] = np.dot( self.a[a_idx].T, delta )
#             print(delta.shape)
            a_idx -= 1
            z_idx -= 1
        
        df_z = self.derivatives['d_'+self.hiddenActivation]( self.z[z_idx] )
        delta = np.dot(delta, self.W[1].T) * df_z
        dJdW[0] = np.dot( x.T, delta )
#         print "actual", dJdW[0].shape
        
        for i in range(len(self.W)):
            self.W[i] = self.W[i] - self.learningRate * dJdW[i]
            
            
    def backward(self, x, y, yHat):
        dJdW = [None] * len(self.W)
        dJdb = [None] * len(self.b)
#         print len(self.W), len(self.a)
        
        a_idx = len(self.a)-1
        z_idx = len(self.z)-1
#         print(y.shape)
#         df_z = self.derivatives['d_'+self.outputActivation]( self.z[z_idx] )
    
        if self.outputLayerSize == 1:
            y = y.reshape(len(y),1)
        
        delta = (yHat-y)
        dJdW[ len(dJdW)-1 ] = np.dot( self.a[a_idx].T, delta )
        
        a_idx -= 1
        z_idx -= 1
#         print("-----")
        for i in range(len(dJdW)-2, 0, -1):
            df_z = self.derivatives['d_'+self.hiddenActivation]( self.z[z_idx] )
            delta = np.multiply( np.dot(delta, self.W[i+1].T), df_z )
            dJdW[i] = np.dot( self.a[a_idx].T, delta )
#             print(delta.shape)
            a_idx -= 1
            z_idx -= 1
        
        df_z = self.derivatives['d_'+self.hiddenActivation]( self.z[z_idx] )
        delta = np.multiply( np.dot(delta, self.W[1].T), df_z )
        dJdW[0] = np.dot( x.T, delta )
#         print "actual", dJdW[0].shape
        
        for i in range(len(self.W)):
#             print(dJdW[i])
            self.W[i] = self.W[i] - self.learningRate * dJdW[i]
            
            
    def fit(self, x, y, num_ephocs=5, batch_size=2000):
        self.y = y
        start = 0
        end = batch_size
        for i in range(num_ephocs):
            while end < x.shape[0]:
                x_batch = x[start:end,:]
                yHat = self.forward(x_batch)
                self.backward(x_batch, y[start:end,:], yHat)
                start += batch_size
                end += batch_size
            x_batch = x[start:x.shape[0],:]
            yHat = self.forward(x_batch)
            self.backward(x_batch, y[start:x.shape[0],:], yHat)

    
    def predict(self, x):
        return self.forward(x)
        
    
    def sigmoid(z):
        return 1.0 / (1 + np.exp(-z))
    
    def relu(z):
        ret = np.maximum(0,z)
        return ret
    
    def tanh(z):
        return np.tanh(z)
    
    def softmax(z):
#         print "<---softmax"
        e_z = np.exp(z - z.max(1).reshape(z.shape[0],1))
#         print "softmax--->"
        ret = e_z / np.sum(e_z,1).reshape(z.shape[0],1)
#         print 'ret', np.sum(ret)
        return ret
    
    def d_sigmoid(z):
        sigmoid_z = 1.0 / (1 + np.exp(-z))
        return sigmoid_z*(1-sigmoid_z)
    
    def d_relu(z):
        ret = copy.deepcopy(z)
        ret[ret <= 0] = 0
        ret[ret > 0] = 1
        return ret
    
    def d_tanh(z):
        tanh_z = np.tanh(z)
        return 1 - (tanh_z)**2
    
    def d_softmax(z):
        e_z = np.exp(z - z.max(1).reshape(z.shape[0],1))
        sm = e_z / np.sum(e_z,1).reshape(z.shape[0],1)
#         sm = sm.reshape(-1,1)
#         return np.diagflat(sm) - np.dot(sm, sm.T)
#         e_z = np.exp(z-np.amax(z))
#         sm = e_z / np.sum(e_z)
        ret = sm*(1-sm)
        return ret
    
    activations ={'sigmoid': sigmoid,
                 'relu': relu,
                 'tanh': tanh,
                 'softmax': softmax
                 }
    derivatives ={'d_sigmoid': d_sigmoid,
                 'd_relu': d_relu,
                 'd_tanh': d_tanh,
                 'd_softmax': d_softmax
                 }

In [257]:
nn = Neural_Network(784,10,1,[128],'tanh','softmax', 0.01)
# print(nn.W)
# X = np.array([ [3,5], [1,10], [7,2] ])
# Y = np.array([0,1,1])
# print X.shape
# print X
# y = nn.forward(X)
# print y
# nn.backward(X, Y, y)
# print nn.W
# print Y.shape
encoder = LabelEncoder()
Y = train_data[Label].values
encoder.fit(Y)
Y = encoder.transform(Y)
Y = one_hot_encode(Y)
# print Y
nn.fit(X, Y, num_ephocs=100, batch_size=100)
# print(nn.W)
op = nn.predict(validation_data[Attributes].values)

Y = validation_data[Label].values
encoder.fit(Y)
Y = encoder.transform(Y)
Y = one_hot_encode(Y)
T, F = 0, 0
for i in range(op.shape[0]):
#     print np.argmax(op[i,:])
#     print op[i]
#     print np.argmax(Y[i])
#     print ""
    if np.argmax(Y[i]) == np.argmax(op[i,:]):
        T += 1
    else:
        F += 1
print "Accuracy = ", float(T)/(T+F)
#     print""

Accuracy =  0.816083333333


In [ ]:
x = np.array( [ [1,9,2], [3,4,5], [10,9,8] ] )
y = np.array( [1,2,3] ).reshape(3,1)
x-y

In [100]:
def reluDerivative(x):
    y=copy.deepcopy(x)
    y[y<=0] = 0
    y[y>0] = 1
    return y

z = np.random.uniform(-1, 1, (3,3))
print z
x = reluDerivative(z)
print x
print z

[[-0.49652977 -0.22665002  0.59535449]
 [-0.89489335  0.33459019  0.50173344]
 [-0.79411626  0.20124216 -0.97674158]]
[[0. 0. 1.]
 [0. 1. 1.]
 [0. 1. 0.]]
[[-0.49652977 -0.22665002  0.59535449]
 [-0.89489335  0.33459019  0.50173344]
 [-0.79411626  0.20124216 -0.97674158]]
